### CPU Inferencing Notebook

#### Install the required packages

In [1]:
import pandas as pd
import re
import torch
import time
from langchain import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain_community.embeddings import SentenceTransformerEmbeddings
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from tqdm import tqdm
from torch import cuda

### Check whether the machine has GPU or not and set the GPU layers

In [2]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

cpu


In [3]:
from accelerate import Accelerator

if cuda.is_available():
    accelerator = Accelerator()
    gpu_layers = 50

else:
    gpu_layers = 0

print(gpu_layers)

0


### Initialize LLM

In [4]:
def initialize_llm(local_llm):
    return LlamaCpp(
        model_path=local_llm,
        temperature=0.3,
        max_tokens=2048,
        top_p=1,
    )

### Initialize Prompt Template

In [5]:
def initialize_prompt_template():
    return """
    Below is a question posed by healthcare professionals
    including nurses, doctors, and researchers in Malawi,
    all of whom actively engage in disease surveillance efforts.

    Context: {context}

    ### Question:
    {question}

    Offer a response that is both accurate and concise, incorporating
    relevant keywords to address the inquiry effectively.
    """

### Initialize Embeddings

In [6]:
def initialize_embeddings():
    return SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

### Initialize Qdrant Client

In [7]:
def initialize_qdrant_client(url):
    return QdrantClient(url=url, prefer_grpc=False)

### Initialize Qdrant Vector Store

In [8]:
def initialize_qdrant_vector_store(client, embeddings):
    return Qdrant(client=client, embeddings=embeddings, collection_name="vector_db")

### Initialize Prompt

In [9]:
def initialize_prompt(prompt_template):
    return PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

### Initialize Retriever

In [10]:
def initialize_retriever(db):
    return db.as_retriever(search_kwargs={"k":1})

### Regex to extract answer and keywords

In [11]:
def regex_to_preprocess_text_from_biomistral(text):
    answer_text = ""
    keywords = []

    # Define patterns for extracting information
    response_pattern = r"### Answer:\n(.*?)\n\n"
    keywords_pattern = r"### Keywords:\n(.*?)\n\n"

    # Extract information using regular expressions
    response_match = re.search(response_pattern, text)
    keywords_match = re.search(keywords_pattern, text)

    # Check if matches were found and extract information
    if response_match:
        answer_text = response_match.group(1)

    if keywords_match:
        keywords = keywords_match.group(1).strip()

    # Return the extracted information
    return answer_text, keywords

### Regex to obtain paragraphs number

In [12]:
def regex_to_obtain_paragraphs_number(source_document):
    paragraphs_number = None

    # Define the regular expression pattern
    paragraphs_number_pattern = r'(\d+)\n'

    # Search for the pattern in the source document
    paragraphs_number_match = re.search(paragraphs_number_pattern, source_document)

    # Extract the paragraph number or page number if the pattern was found
    if paragraphs_number_match:
        paragraphs_number = paragraphs_number_match.group(1)

    # Return the extracted paragraph number
    return paragraphs_number

### Regex to obtain file name and get booklets

In [13]:
def regex_to_obtain_file_name_and_get_booklet(doc):
    # Define the regular expression pattern
    pattern = r'([^/]+)$'

    # Search for the pattern in the text
    matches = re.search(pattern, doc)

    # Extract the file name if found
    if matches:
        file_name = matches.group(1)

    file_booklet_name = {
        "TG_Booklet_1.xlsx": "TG Booklet 1",
        "TG_Booklet_2.xlsx": "TG Booklet 2",
        "TG_Booklet_3.xlsx": "TG Booklet 3",
        "TG_Booklet_4.xlsx": "TG Booklet 4",
        "TG_Booklet_5.xlsx": "TG Booklet 5",
        "TG_Booklet_6.xlsx": "TG Booklet 6"
    }

    # Return the correspondent booklet name based on the file name
    return file_booklet_name[file_name]

In [14]:
local_llm = "../biomistral-quantized-version/biomistral-finetuned-7b-v2.1-8-bit-gguf-unsloth.Q8_0.gguf"
test_csv = "./Test.csv"
llm = initialize_llm(local_llm)
print("LLM Initialized....")
test_df = pd.read_csv(test_csv)
print("Test Data Loaded....")

prompt_template = initialize_prompt_template()
embeddings = initialize_embeddings()

url = "http://localhost:6333"
client = initialize_qdrant_client(url)

db = initialize_qdrant_vector_store(client, embeddings)
prompt = initialize_prompt(prompt_template)
retriever = initialize_retriever(db)

chain_type_kwargs = {"prompt": prompt}
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True, chain_type_kwargs=chain_type_kwargs, verbose=True)


results = []
keywords = []
sources = []

total_rows = len(test_df)
batch_size = 298
num_batches = (total_rows + batch_size - 1) // batch_size


for batch_index in range(num_batches):
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, total_rows)
    batch_df = test_df[start_index:end_index]

    for index, row in tqdm(batch_df.iterrows(), total=len(batch_df)):
        try:
            query = row['Question Text']
            response = qa(query)
            answer = response['result']
            answer_, keywords_ = regex_to_preprocess_text_from_biomistral(answer)
            answer_ = answer_.strip()
            source_document = response['source_documents'][0].page_content
            doc = response['source_documents'][0].metadata['source'].split("/")[-1]
            results.append(answer_)
            keywords.append(keywords_)
            sources.append(doc)
        except:
            results.append("Error")
            keywords.append("Error")
            sources.append("Error")

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./biomistral-quantized-version/biomistral-finetuned-7b-v2.1-gguf-unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Adeptschneider
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader:

LLM Initialized....
Test Data Loaded....


  0%|          | 0/298 [00:00<?, ?it/s]/home/adeptschneiderthedev/.miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...



llama_print_timings:        load time =    2183.06 ms
llama_print_timings:      sample time =     170.65 ms /   260 runs   (    0.66 ms per token,  1523.63 tokens per second)
llama_print_timings: prompt eval time =   73046.75 ms /   252 tokens (  289.87 ms per token,     3.45 tokens per second)
llama_print_timings:        eval time =  103726.63 ms /   259 runs   (  400.49 ms per token,     2.50 tokens per second)
llama_print_timings:       total time =  178088.87 ms /   511 tokens
  0%|          | 1/298 [02:58<14:45:21, 178.86s/it]Llama.generate: prefix-match hit


...


> Finished chain.


> Entering new RetrievalQA chain...



llama_print_timings:        load time =    2183.06 ms
llama_print_timings:      sample time =     105.85 ms /   176 runs   (    0.60 ms per token,  1662.73 tokens per second)
llama_print_timings: prompt eval time =   60095.90 ms /   240 tokens (  250.40 ms per token,     3.99 tokens per second)
llama_print_timings:        eval time =   55869.44 ms /   175 runs   (  319.25 ms per token,     3.13 tokens per second)
llama_print_timings:       total time =  116808.17 ms /   415 tokens
100%|█████████▉| 200/201 [7:15:18<01:59, 119.97s/it]


> Finished chain.


> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit

llama_print_timings:        load time =    2183.06 ms
llama_print_timings:      sample time =     124.43 ms /   204 runs   (    0.61 ms per token,  1639.42 tokens per second)
llama_print_timings: prompt eval time =   62774.09 ms /   261 tokens (  240.51 ms per token,     4.16 tokens per second)
llama_print_timings:        eval time =   65510.24 ms /   203 runs   (  322.71 ms per token,     3.10 tokens per second)
llama_print_timings:       total time =  129286.71 ms /   464 tokens
100%|██████████| 201/201 [7:17:27<00:00, 130.59s/it]


> Finished chain.


In [15]:
mysub = test_df.copy()
mysub["Answers"] = results
mysub["Keywords"] = keywords
mysub["Source files"] = sources
mysub.to_csv("full_test.csv", index=False)

#### Extracting Keywords and Paragraph

In [16]:
test_set = pd.read_csv("full_test.csv")
test_set

,ID,Question Text,Answers,Source files
0,Q4,"What is the definition of ""unusual event""",An unusual event is defined as any event that ...,TG_Booklet_1.xlsx
1,Q5,What is Community Based Surveillance (CBS)?,CBS is the process of collecting and reporting...,TG_Booklet_1.xlsx
2,Q9,What kind of training should members of VHC re...,Members of VHC should receive training on how ...,TG_Booklet_1.xlsx
3,Q10,What is indicator based surveillance (IBS)?,Indicator based surveillance (IBS) is the stan...,TG_Booklet_1.xlsx
4,Q13,What is Case based surveillance?,Case-based surveillance involves the ongoing a...,TG_Booklet_1.xlsx
...,...,...,...,...
494,Q1229,Where should completeness be evaluated in the ...,Completeness should be evaluated at the nation...,TG_Booklet_4.xlsx
495,Q1230,Which dimensions of completeness are crucial i...,The crucial dimensions of completeeness in sur...,TG_Booklet_4.xlsx
496,Q1236,How can the completeness of case reporting be ...,The completeness of case reporting can be moni...,TG_Booklet_4.xlsx
497,Q1239,Where should completeness and timeliness of re...,Completeeness and timeliness of reports should...,TG_Booklet_4.xlsx


In [17]:
!pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources (run only once)
nltk.download('punkt')
nltk.download('stopwords')


def extract_keywords(provided_text):
    # Tokenize the text
    tokens = word_tokenize(provided_text)

    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token.title() for token in tokens if token not in stop_words]

    # Remove punctuation and non-alphabetic characters
    keywords = [token for token in filtered_tokens if token.isalpha()]

    # Remove duplicate keywords
    unique_keywords = list(set(keywords))

    return ', '.join(unique_keywords)





def find_matching_paragraphs(csv_filepath, text_to_check, threshold=0.9):
    # Load the DataFrame
    data_dir = os.path.join(".", "data", "MWTGBookletsExcel")
    file_path = os.path.join(data_dir, csv_filepath)
    df = pd.read_excel(file_path, names=["paragraph", "text"])
    df.fillna('', inplace=True)
    # Concatenate all text from the 'text' column in the DataFrame
    all_text = ' '.join(df['text'].astype(str).values.tolist())

    # Combine the provided text and all text from the DataFrame
    combined_text = [text_to_check, all_text]

    # Initialize TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit and transform the text in the DataFrame
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

    # Transform the provided text
    provided_text_tfidf = tfidf_vectorizer.transform([text_to_check])

    # Calculate cosine similarity between the provided text and each paragraph in the DataFrame
    cosine_similarities = cosine_similarity(provided_text_tfidf, tfidf_matrix).flatten()

    # Find paragraphs that meet or exceed the threshold
    matching_paragraph_indices = [i for i, score in enumerate(cosine_similarities) if score >= threshold]

    if matching_paragraph_indices:
        # Get the corresponding paragraph numbers
        matching_paragraph_numbers = df.iloc[matching_paragraph_indices]['paragraph'].tolist()
        matching_paragraph_numbers = [str(int(i)) for i in matching_paragraph_numbers]
        return ', '.join(matching_paragraph_numbers)

    else:
        # If no paragraphs meet the threshold, fallback to selecting the paragraph with the highest similarity
        closest_paragraph_index = cosine_similarities.argmax()
        closest_paragraph_number = df.iloc[closest_paragraph_index]['paragraph']
        return ', '.join([str(closest_paragraph_number)])

[nltk_data] Downloading package punkt to
[nltk_data]     /home/adeptschneiderthedev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/adeptschneiderthedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
test_set[test_set["Answers"] == "Error"]

,ID,Question Text,Answers,Source files
244,Q584,"Does the TG mention CBIS as an abbreviation, a...",Error,Error
247,Q590,"Is there an abbreviation CHT in the TG, and wh...",Error,Error
248,Q593,"What does DDO stand for, and is it explained i...",Error,Error
250,Q597,"Does the TG mention DPC as an abbreviation, an...",Error,Error
252,Q606,"Is HCF an abbreviation mentioned in the TG, an...",Error,Error
254,Q613,"Does the TG include IBS as an abbreviation, an...",Error,Error
255,Q615,"Is IEC an abbreviation mentioned in the TG, an...",Error,Error
261,Q628,"What does MOA stand for, and is it explained i...",Error,Error
262,Q629,"Is MTI an abbreviation mentioned in the TG, an...",Error,Error
264,Q631,"Is NNT an abbreviation mentioned in the TG, an...",Error,Error


In [28]:
# Check for NaN values in the 'Answers' column and replace them with 'Target not found'
test_set['Answers'].fillna('Target not found', inplace=True)

In [30]:
# Check for unique values in Source files column
test_set['Source files'].unique()

array(['TG_Booklet_1.xlsx', 'TG_Booklet_2.xlsx', 'TG_Booklet_6.xlsx',
       'TG_Booklet_4.xlsx', 'TG_Booklet_3.xlsx', 'TG_Booklet_5.xlsx',
       'Error'], dtype=object)

In [31]:
ID = []
Target = []

for index, row in tqdm(test_set.iterrows(), total=len(test_set)):
    if row["Answers"]== "Error":
        ID.append(row["ID"]+"_keywords")
        Target.append(extract_keywords(row["Question Text"]))
        ID.append(row["ID"]+"_paragraph(s)_number")
        Target.append(find_matching_paragraphs("TG_Booklet_1.xlsx", row["Question Text"], threshold=0.9))
        ID.append(row["ID"]+"_question_answer")
        Target.append(" ")
        ID.append(row["ID"]+"_reference_document")
        Target.append("TG Booklet 1")

    else:
        ID.append(row["ID"]+"_keywords")
        Target.append(row["Keywords"])
        ID.append(row["ID"]+"_paragraph(s)_number")
        Target.append(find_matching_paragraphs(row["Source files"], row["Answers"], threshold=0.9))
        ID.append(row["ID"]+"_question_answer")
        Target.append(row["Answers"])
        ID.append(row["ID"]+"_reference_document")
        Target.append(row["Source files"].split(".xlsx")[0])

100%|██████████| 499/499 [00:56<00:00,  8.88it/s]


In [32]:
ss = pd.read_csv("./SampleSubmission.csv")
ss

,ID,Target
0,Q1000_keywords,
1,Q1000_paragraph(s)_number,
2,Q1000_question_answer,
3,Q1000_reference_document,
4,Q1002_keywords,
...,...,...
1991,Q999_reference_document,
1992,Q9_keywords,
1993,Q9_paragraph(s)_number,
1994,Q9_question_answer,


In [33]:
ss["ID"] = ID
ss["Target"] = Target
ss["Target"] = ss["Target"].fillna(" ")

In [34]:
# Look for Target values with TG_Booklet_1, TG_Booklet_2, TG_Booklet_3, TG_Booklet_4, TG_Booklet_5, TG_Booklet_6 and replace them with TG Booklet 1, TG Booklet 2, TG Booklet 3, TG Booklet 4, TG Booklet 5, TG Booklet 6
ss["Target"] = ss["Target"].str.replace("TG_Booklet_1", "TG Booklet 1")
ss["Target"] = ss["Target"].str.replace("TG_Booklet_2", "TG Booklet 2")
ss["Target"] = ss["Target"].str.replace("TG_Booklet_3", "TG Booklet 3")
ss["Target"] = ss["Target"].str.replace("TG_Booklet_4", "TG Booklet 4")
ss["Target"] = ss["Target"].str.replace("TG_Booklet_5", "TG Booklet 5")
ss["Target"] = ss["Target"].str.replace("TG_Booklet_6", "TG Booklet 6")
ss.to_csv("cpu_submission.csv", index=False)

In [ ]:
ss